# FLIP (01) Practical Data Science

---
Team Director: Shaoni Wang | snwang@tulip.academy<br />

TULIP Academy <br />
http://www.tulip.academy

---
## Session 53 Frequent Pattern Mining
---

Mining frequent items, itemsets, subsequences, or other substructures is usually among the first steps to analyze a large-scale dataset, which has been an active research topic in data mining for years. We refer users to Wikipedia’s [association rule learning](http://en.wikipedia.org/wiki/Association_rule_learning) for more information. spark.mllib provides a parallel implementation of FP-growth, a popular algorithm to mining frequent itemsets. 



### Part 1  Frequent Patterns Mining 

1.1 [FP-growth](#fp)



---
<a id = "fp"></a>
## <span style="color:#0b486b">1. FP-growth</span>


The FP-growth algorithm is described in the paper [Han et al., Mining frequent patterns without candidate generation](http://dx.doi.org/10.1145/335191.335372), where “FP” stands for frequent pattern. Given a dataset of transactions, the first step of FP-growth is to calculate item frequencies and identify frequent items. Different from [Apriori-like](http://en.wikipedia.org/wiki/Apriori_algorithm) algorithms designed for the same purpose, the second step of FP-growth uses a suffix tree (FP-tree) structure to encode transactions without generating candidate sets explicitly, which are usually expensive to generate. After the second step, the frequent itemsets can be extracted from the FP-tree. In spark.mllib, we implemented a parallel version of FP-growth called PFP, as described in [Li et al., PFP: Parallel FP-growth for query recommendation](http://dx.doi.org/10.1145/1454008.1454027). PFP distributes the work of growing FP-trees based on the suffices of transactions, and hence more scalable than a single-machine implementation. We refer users to the papers for more details.

spark.mllib’s FP-growth implementation takes the following (hyper-)parameters:

* minSupport: the minimum support for an itemset to be identified as frequent. For example, if an item appears 3 out of 5 transactions, it has a support of 3/5=0.6.
* numPartitions: the number of partitions used to distribute the work.

[FPGrowth](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.fpm.FPGrowth) implements the FP-growth algorithm. It take an RDD of transactions, where each transaction is an List of items of a generic type. Calling FPGrowth.train with transactions returns an [FPGrowthModel](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.fpm.FPGrowthModel) that stores the frequent itemsets with their frequencies.

Refer to the [FPGrowth Python docs](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.fpm.FPGrowth) for more details on the API.

Assume that the file `sample_fqgrowth.txt` has been uploaded into DBFS and noted down the address.

In [ ]:
from pyspark.mllib.fpm import FPGrowth

data = sc.textFile("/FileStore/tables/387gaqza1491403501002/sample_fpgrowth.txt") # replace with data file path

transactions = data.map(lambda line: line.strip().split(' '))
model = FPGrowth.train(transactions, minSupport=0.2, numPartitions=10)
result = model.freqItemsets().collect()
for fi in result:
    print(fi)

Find full example code at "examples/src/main/python/mllib/fpgrowth_example.py" in the Spark repo.